In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import numpy as np
import pandas as pd

batch_size = 64
num_classes = 10
epochs = 100
target_val_acc=0.9940

Using TensorFlow backend.


In [0]:
#!unzip  -q '/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/train.csv.zip' -d '/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/train.csv'

In [0]:
# input image dimensions.
img_rows, img_cols = 28, 28


df = pd.read_csv('/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/train.csv')
x_train_df = df.drop(['label'], 1, inplace=False)
y_train_df = df['label']

x_train = np.array(x_train_df.values.flatten()).reshape(len(x_train_df), img_rows, img_cols, 1)
y_train = y_train_df.tolist()
input_shape = (img_rows, img_cols, 1)

In [5]:
print(input_shape)

(28, 28, 1)


In [6]:
#Normalization.
x_train = x_train.astype('float32')
x_train/= 255
print('Original dataset:')
print('x_train shape:', x_train.shape)


Original dataset:
x_train shape: (42000, 28, 28, 1)


In [7]:
#Generates augmented images from the x_train images 
#array (and uses corresponding y_train labels ofc).
datagen = ImageDataGenerator(rotation_range=20,
                            width_shift_range=0.10,
                            height_shift_range=0.10,
                            zoom_range=[0.8, 1.2],
                            shear_range=0.2)

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True)

#Appends augmented images to the original x_train and y_train arrays.

deg=1            #deg controls the extent of the augmentation. 1 indicates
                 #whole 42k images will be yielded in batches and used for
                 #training hence increasing train set to ~84k images.

print("Pre-processing images for data augmentation...Wait a minute...")

iterate=(int)((42000/batch_size)*deg)

for i in range(0,iterate):
    ret=train_gen.next()  
    x_train=np.append(x_train,ret[0],axis=0)
    y_train=np.append(y_train,ret[1])

print("Done adding!")
print(x_train.shape[0], 'Total samples')

Pre-processing images for data augmentation...Wait a minute...
Done adding!
83984 Total samples


In [8]:
x_val = x_train[:23984,:,:,:]
x_train = x_train[23984:,:,:,:]
y_val = y_train[:23984]
y_train = y_train[23984:]
print(x_train.shape[0], 'Train samples')
print(x_val.shape[0], 'val samples')

60000 Train samples
23984 val samples


In [0]:
#Convert labels to one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

In [0]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 0.001
  else:
    return 0.001 * np.exp(0.1 * (40 - epoch))

#Callback for lr updation and stop training 
class myCallback(keras.callbacks.LearningRateScheduler):
    def __init__(self, schedule, val_target):
        super(myCallback, self).__init__(schedule)     
        self.val_target=val_target

    def on_epoch_end(self, epoch, logs={}):
        try:
            if logs.get('val_acc')>=self.val_target:
                self.model.stop_training = True
                print('Stopped training as target val_acc reached!')
            else:
                lr_=float(keras.backend.get_value(self.model.optimizer.lr))
                print('lr: ','{:.8f}'.format(lr_)) 
        except:
                lr_=float(keras.backend.get_value(self.model.optimizer.lr))
                print('lr: ','{:.8f}'.format(lr_)) 

    
callback0 = myCallback(scheduler,target_val_acc)

In [11]:
#Model
model = Sequential()

model.add(Conv2D(8, kernel_size=(3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Conv2D(8, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(14, (3, 3),activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Conv2D(14, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

#Training
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val),
          callbacks=[callback0])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 8)         584       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 8)         584       
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 14)          1022      
______________________________________

In [12]:
#validation Testing
score = model.evaluate(x_val, y_val, verbose=0)
print('val loss:', score[0])
print('val accuracy:', score[1])

val loss: 0.027904890140992594
val accuracy: 0.9918279051780701


In [0]:
#model.predict(x_test,verbose=0)

In [0]:
df = pd.read_csv('/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/test.csv')
df=df.astype('float32')/255.0

In [15]:
x_test = np.array(df.values.flatten()).reshape(len(df), img_rows, img_cols,1)
t=model.predict_classes(x_test,verbose=1)
t

28000/28000 [==============================] - 2s 75us/step


array([2, 0, 9, ..., 3, 9, 2])

In [0]:
dfsubmit = pd.read_csv('/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/gs.csv')    
dfsubmit['Label']=t
dfsubmit
dfsubmit.to_csv('/content/drive/My Drive/ML Enthusiasts/Vidit/codeML/MNIST/result.csv',header=True, index=False)